## Use the API to create some test data


In [ ]:
# Local Docker
auth_url = 'http://web:8000/api-token-auth/'
participant_url = 'http://web:8000/participant/'
submission_url = 'http://web:8000/submission/'

In [34]:
# AWS (demo)
auth_url = 'https://researchnet.ictedge.org/api-token-auth/'
participant_url = 'https://researchnet.ictedge.org/participant/'
submission_url = 'https://researchnet.ictedge.org/submission/'

In [21]:
# Local
auth_url = 'http://127.0.0.1:8000/api-token-auth/'
participant_url = 'http://127.0.0.1:8000/participant/'
submission_url = 'http://127.0.0.1:8000/submission/'

In [35]:
import requests
payload = {'username': 'researcher', 'password':'1234thumbwar'}
r = requests.post(auth_url, json=payload)


In [36]:
r.json()['token']

'9a1194980e9e15da451d9ab68a5b8801e6704c18'

### Make up some stuff

In [26]:
import random

first_names = ['Pat', 'Jessie', 'Same', 'Sue', 'Chris', 'Piper']
last_names = ['Smith', 'Jones', 'Roberts', 'Walters', 'Thomas']
gender_options = ['Male', 'Female']

# lat/long
fake_places = [
     ("40.5895466", "-105.0751243"),
     ("41.5895466", "-104.0751243"),
     ("39.7640021","-105.1353041"),
     ("38.7640021","-104.1353041"),
     ("39.7724466","-105.9249464"),
     ("35.7724466","-103.9249464"),
     ("39.1490189","-107.1444289"),
     ("38.1490189","-106.1444289"),
     ("36.0017455","-79.0249976"),
     ("37.0017455","-80.0249976"),
     ("35.9661857","-79.1067084"),
     ("34.9661857","-78.1067084"),
     ("45.5423508","-122.7945071")
    ]


In [27]:
from random import randrange
from datetime import timedelta, datetime

def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [28]:
d1 = datetime.strptime('1/1/1967', '%m/%d/%Y')
d2 = datetime.strptime('1/1/1987', '%m/%d/%Y')

## Load Enrollments
The step step here is to generate some sample data.  Feel free to adjust this as needed.

In [29]:

r_auth = requests.post(auth_url, json=payload)
auth_token = r_auth.json()['token']

In [30]:
# Get the token

auth_token = r.json()['token']
headers = {'Authorization': 'Token '+auth_token }

for i in range(10):
    random_birthday = random_date(d1, d2)
    payload = {'username': 'testuser'+str(i), 
               'password':'testpassword', 
               'first_name': random.choice(first_names),
               'last_name': random.choice(last_names),
               'email': 'fakeemail@rti.org',
               'dob': str(random_birthday.strftime('%Y-%m-%d')),
               'gender': random.choice(gender_options),
              }
    
    r = requests.post(participant_url, json=payload, headers=headers)
    if r.status_code != 201:
        print("there is a problem here: " + str(r.text))

## Load Submissions

In [31]:
# Randomize test data (its more interesting this way)
moods_category = ['good', 'bad', 'fine', 'unsure', 'no response', 'asleep']
mood_scale = ['1','2','3','4','5','6','7','8','9','0']
mood_image = ['1','2','3','4','5','6','7','8','9','0']

In [32]:
# Define start and end date
d1 = datetime.strptime('1/1/2016', '%m/%d/%Y')  
d2 = datetime.strptime('6/1/2016', '%m/%d/%Y')

In [33]:
import random
import geocoder

# loop through the same participants loaded earlier
for i in range(10):
    
    # Get the token
    auth_payload = {'username': 'testuser'+str(i), 'password':'testpassword'}
    r_auth = requests.post(auth_url, json=auth_payload)
    auth_token = r_auth.json()['token']
    

    # For each user, give a random number of submissions up to 10
    for y in range(random.randrange(1, 10)):
        
        random_time = random_date(d1, d2)
        fake_place = random.choice(fake_places)
        g = geocoder.google([fake_place[0], fake_place[1]], method='reverse')

        headers = {'Authorization': 'Token '+auth_token }
        
        # use this format: YYYY-MM-DDThh:mm for timestamp
        submission_payload = {
            'device_id': 'TEST-49D5-4CAD-AE42-E5CE922A3346', 
            'time_start': str(random_time.strftime('%Y-%m-%d %T')), 
            'time_complete': str(random_time.strftime('%Y-%m-%d %T')),
            'lat' : fake_place[0],
           'long': fake_place[1],   
           'place': g.city+", "+g.state +" "+ g.country,                 
           'response': { 
                'mood cateogry': random.choice(moods_category),
                'mood scale': random.choice(mood_scale),
                'mood image': random.choice(mood_image)
            }
        }
     
        r_sub = requests.post(submission_url, json=submission_payload, headers=headers)


In [31]:
# YYYY-MM-DDThh:mm for timestamp %H:%M:%S +0000
# this works: 2016-03-30 14:54:17.313583-06
random_time.strftime('%Y-%m-%d %T')

'2016-02-08 02:42:28'

# A note about timestamp
if you want to update the timestamp, you'll have to run a seperate sql update: "update core_submission set timestamp = time_start;". We intentionally have this set as the time that the record was inserted. The timestart and time complete come from the device

In [26]:
submission_payload

{'device_id': 'TEST-49D5-4CAD-AE42-E5CE922A3346',
 'lat': '45.5423508',
 'long': '-122.7945071',
 'place': 'Portland, OR US',
 'response': {'mood cateogry': 'bad', 'mood image': '2', 'mood scale': '8'},
 'time_complete': '2016-02-08-02:42:28',
 'time_start': '2016-02-08-02:42:28',
 'timestamp': '2016-02-08-02:42:28'}

In [25]:
r_sub.text

'{"time_complete":["Datetime has wrong format. Use one of these formats instead: YYYY-MM-DDThh:mm[:ss[.uuuuuu]][+HH:MM|-HH:MM|Z]."],"time_start":["Datetime has wrong format. Use one of these formats instead: YYYY-MM-DDThh:mm[:ss[.uuuuuu]][+HH:MM|-HH:MM|Z]."]}'

In [67]:
submission_payload

{'device_id': 'TEST-49D5-4CAD-AE42-E5CE922A3346',
 'lat': '37.0017455',
 'long': '-80.0249976',
 'place': 'Callaway, VA US',
 'response': {'mood cateogry': 'no response',
  'mood image': '4',
  'mood scale': '6'}}

In [57]:
g.city

In [62]:

import geocoder
# 44.5423508 and -121.7945071
# 34.8436867 and -75.7851426
# 40.5895466, -105.0751243 fort collins
g = geocoder.google([34.8436867, -75.7851426], method='reverse')
g.city


In [33]:
for i in range(2):
    fake = random.choice(fake_places)
    g = geocoder.google([fake[0], fake[1]], method='reverse')
    print("doing: " + fake[0] +" and " + fake[1])
    print(g.city+", "+g.state +" "+ g.country)
    

doing: 36.0017455 and -79.0249976
Durham, NC US
doing: 39.7640021 and -105.1353041
Wheat Ridge, CO US


In [15]:
name = "adam"
last = "preston"
print(name+" "+last)

adam preston


In [22]:
print(g.country)

US


In [25]:
 test = random.choice(fake_places)

In [26]:
test[1]

'-104.0751243'